In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/andro.arff')
data = pd.DataFrame(data)
data.head()

,Window0-Att0,Window0-Att1,Window0-Att2,Window0-Att3,Window0-Att4,Window0-Att5,Window1-Att0,Window1-Att1,Window1-Att2,Window1-Att3,...,Window4-Att2,Window4-Att3,Window4-Att4,Window4-Att5,Target,Target_2,Target_3,Target_4,Target_5,Target_6
0,11.84,5.08,49.0,32.2,16.5,1.40,12.01,5.09,48.0,32.0,...,45.0,29.7,38.5,3.27,15.17,5.14,46.0,30.4,46.8,3.74
1,12.01,5.09,48.0,32.0,14.8,1.25,12.89,5.10,48.0,31.9,...,46.0,30.1,42.6,3.55,14.85,5.11,47.0,31.2,46.8,3.78
2,12.89,5.10,48.0,31.9,8.6,0.71,13.95,5.11,47.0,30.8,...,48.0,31.3,46.0,3.79,14.72,5.10,48.0,31.7,42.7,3.40
3,13.95,5.11,47.0,30.8,20.2,1.64,13.75,5.13,45.0,29.7,...,47.0,31.1,46.7,3.79,15.28,5.11,48.0,32.0,42.5,3.38
4,13.75,5.13,45.0,29.7,38.5,3.27,13.34,5.13,46.0,30.1,...,45.0,29.7,45.1,3.75,16.15,5.13,47.0,30.8,42.1,3.27


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(49, 30) (49, 6)


In [4]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(34, 30) (34, 6) (15, 30) (15, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 49
    p = 30

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### Bidirectional Model

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 2

In [8]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

### Forward Ensemble

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (34, 30) 
 Y = (34, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
34/34 [==============================] - 0s - loss: 0.6999 - mean_squared_error: 0.6999
Epoch 2/500
34/34 [==============================] - 0s - loss: 0.4758 - mean_squared_error: 0.4758
Epoch 3/500
34/34 [==============================] - 0s - loss: 0.3336 - mean_squared_error: 0.3336
Epoch 4/500
34/34 [==============================] - 0s - loss: 0.2556 - mean_squared_error: 0.2556
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.2244 - mean_squared_error: 0.2244
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.2127 - mean_squared_error: 0.2127
Epoch 7/500
34/34 [==============================] - 0s - loss: 0.2001 - mean_squared_error: 0.2001
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.1805 - mean_squared_error

34/34 [==============================] - 0s - loss: 2.3475e-04 - mean_squared_error: 2.3475e-04
Epoch 66/500
34/34 [==============================] - 0s - loss: 2.2479e-04 - mean_squared_error: 2.2479e-04
Epoch 67/500
34/34 [==============================] - 0s - loss: 2.3191e-04 - mean_squared_error: 2.3191e-04
Epoch 68/500
34/34 [==============================] - 0s - loss: 2.3035e-04 - mean_squared_error: 2.3035e-04
Epoch 69/500
34/34 [==============================] - 0s - loss: 2.1202e-04 - mean_squared_error: 2.1202e-04
Epoch 70/500
34/34 [==============================] - 0s - loss: 1.8845e-04 - mean_squared_error: 1.8845e-04
Epoch 71/500
34/34 [==============================] - 0s - loss: 1.7179e-04 - mean_squared_error: 1.7179e-04
Epoch 72/500
34/34 [==============================] - 0s - loss: 1.5966e-04 - mean_squared_error: 1.5966e-04
Epoch 73/500
34/34 [==============================] - 0s - loss: 1.4291e-04 - mean_squared_error: 1.4291e-04
Epoch 74/500
34/34 [============

34/34 [==============================] - 0s - loss: 1.3391e-07 - mean_squared_error: 1.3391e-07
Epoch 140/500
34/34 [==============================] - 0s - loss: 1.0931e-07 - mean_squared_error: 1.0931e-07
Epoch 141/500
34/34 [==============================] - 0s - loss: 9.7634e-08 - mean_squared_error: 9.7634e-08
Epoch 142/500
34/34 [==============================] - 0s - loss: 8.7981e-08 - mean_squared_error: 8.7981e-08
Epoch 143/500
34/34 [==============================] - 0s - loss: 6.8463e-08 - mean_squared_error: 6.8463e-08
Epoch 144/500
34/34 [==============================] - 0s - loss: 4.8735e-08 - mean_squared_error: 4.8735e-08
Epoch 145/500
34/34 [==============================] - 0s - loss: 4.4216e-08 - mean_squared_error: 4.4216e-08
Epoch 146/500
34/34 [==============================] - 0s - loss: 5.2081e-08 - mean_squared_error: 5.2081e-08
Epoch 147/500
34/34 [==============================] - 0s - loss: 5.4447e-08 - mean_squared_error: 5.4447e-08
Epoch 148/500
34/34 [===

34/34 [==============================] - 0s - loss: 3.9437e-11 - mean_squared_error: 3.9437e-11
Epoch 214/500
34/34 [==============================] - 0s - loss: 3.8427e-11 - mean_squared_error: 3.8427e-11
Epoch 215/500
34/34 [==============================] - 0s - loss: 4.2197e-11 - mean_squared_error: 4.2197e-11
Epoch 216/500
34/34 [==============================] - 0s - loss: 4.1647e-11 - mean_squared_error: 4.1647e-11
Epoch 217/500
34/34 [==============================] - 0s - loss: 3.6035e-11 - mean_squared_error: 3.6035e-11
Epoch 218/500
34/34 [==============================] - 0s - loss: 3.2931e-11 - mean_squared_error: 3.2931e-11
Epoch 219/500
34/34 [==============================] - 0s - loss: 3.2956e-11 - mean_squared_error: 3.2956e-11
Epoch 220/500
34/34 [==============================] - 0s - loss: 2.9640e-11 - mean_squared_error: 2.9640e-11
Epoch 221/500
34/34 [==============================] - 0s - loss: 2.5556e-11 - mean_squared_error: 2.5556e-11
Epoch 222/500
34/34 [===

34/34 [==============================] - 0s - loss: 3.2359e-14 - mean_squared_error: 3.2359e-14
Epoch 288/500
34/34 [==============================] - 0s - loss: 3.0216e-14 - mean_squared_error: 3.0216e-14
Epoch 289/500
34/34 [==============================] - 0s - loss: 3.0852e-14 - mean_squared_error: 3.0852e-14
Epoch 290/500
34/34 [==============================] - 0s - loss: 2.2897e-14 - mean_squared_error: 2.2897e-14
Epoch 291/500
34/34 [==============================] - 0s - loss: 2.4731e-14 - mean_squared_error: 2.4731e-14
Epoch 292/500
34/34 [==============================] - 0s - loss: 2.3960e-14 - mean_squared_error: 2.3960e-14
Epoch 293/500
34/34 [==============================] - 0s - loss: 2.5841e-14 - mean_squared_error: 2.5841e-14
Epoch 294/500
34/34 [==============================] - 0s - loss: 2.2853e-14 - mean_squared_error: 2.2853e-14
Epoch 295/500
34/34 [==============================] - 0s - loss: 1.5705e-14 - mean_squared_error: 1.5705e-14
Epoch 296/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 2.7311e-07 - mean_squared_error: 2.7311e-07
Epoch 192/500
34/34 [==============================] - 0s - loss: 2.5579e-07 - mean_squared_error: 2.5579e-07
Epoch 193/500
34/34 [==============================] - 0s - loss: 2.3952e-07 - mean_squared_error: 2.3952e-07
Epoch 194/500
34/34 [==============================] - 0s - loss: 2.2445e-07 - mean_squared_error: 2.2445e-07
Epoch 195/500
34/34 [==============================] - 0s - loss: 2.1034e-07 - mean_squared_error: 2.1034e-07
Epoch 196/500
34/34 [==============================] - 0s - loss: 1.9746e-07 - mean_squared_error: 1.9746e-07
Epoch 197/500
34/34 [==============================] - 0s - loss: 1.8494e-07 - mean_squared_error: 1.8494e-07
Epoch 198/500
34/34 [==============================] - 0s - loss: 1.7338e-07 - mean_squared_error: 1.7338e-07
Epoch 199/500
34/34 [==============================] - 0s - loss: 1.6237e-07 - mean_squared_error: 1.6237e-07
Epoch 200/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.0056e-09 - mean_squared_error: 1.0056e-09
Epoch 266/500
34/34 [==============================] - 0s - loss: 9.1469e-10 - mean_squared_error: 9.1469e-10
Epoch 267/500
34/34 [==============================] - 0s - loss: 8.3501e-10 - mean_squared_error: 8.3501e-10
Epoch 268/500
34/34 [==============================] - 0s - loss: 7.6074e-10 - mean_squared_error: 7.6074e-10
Epoch 269/500
34/34 [==============================] - 0s - loss: 6.8997e-10 - mean_squared_error: 6.8997e-10
Epoch 270/500
34/34 [==============================] - 0s - loss: 6.3119e-10 - mean_squared_error: 6.3119e-10
Epoch 271/500
34/34 [==============================] - 0s - loss: 5.7218e-10 - mean_squared_error: 5.7218e-10
Epoch 272/500
34/34 [==============================] - 0s - loss: 5.2060e-10 - mean_squared_error: 5.2060e-10
Epoch 273/500
34/34 [==============================] - 0s - loss: 4.7461e-10 - mean_squared_error: 4.7461e-10
Epoch 274/500
34/34 [===

34/34 [==============================] - 0s - loss: 7.1712e-13 - mean_squared_error: 7.1712e-13
Epoch 340/500
34/34 [==============================] - 0s - loss: 6.8743e-13 - mean_squared_error: 6.8743e-13
Epoch 341/500
34/34 [==============================] - 0s - loss: 6.5353e-13 - mean_squared_error: 6.5353e-13
Epoch 342/500
34/34 [==============================] - 0s - loss: 6.3202e-13 - mean_squared_error: 6.3202e-13
Epoch 343/500
34/34 [==============================] - 0s - loss: 5.2851e-13 - mean_squared_error: 5.2851e-13
Epoch 344/500
34/34 [==============================] - 0s - loss: 4.6951e-13 - mean_squared_error: 4.6951e-13
Epoch 345/500
34/34 [==============================] - 0s - loss: 4.4695e-13 - mean_squared_error: 4.4695e-13
Epoch 346/500
34/34 [==============================] - 0s - loss: 5.0577e-13 - mean_squared_error: 5.0577e-13
Epoch 347/500
34/34 [==============================] - 0s - loss: 3.8027e-13 - mean_squared_error: 3.8027e-13
Epoch 348/500
34/34 [===

34/34 [==============================] - 0s - loss: 8.5613e-14 - mean_squared_error: 8.5613e-14
Epoch 414/500
34/34 [==============================] - 0s - loss: 8.0565e-14 - mean_squared_error: 8.0565e-14
Epoch 415/500
34/34 [==============================] - 0s - loss: 7.5063e-14 - mean_squared_error: 7.5063e-14
Epoch 416/500
34/34 [==============================] - 0s - loss: 8.1581e-14 - mean_squared_error: 8.1581e-14
Epoch 417/500
34/34 [==============================] - 0s - loss: 7.1700e-14 - mean_squared_error: 7.1700e-14
Epoch 418/500
34/34 [==============================] - 0s - loss: 7.0962e-14 - mean_squared_error: 7.0962e-14
Epoch 419/500
34/34 [==============================] - 0s - loss: 8.0425e-14 - mean_squared_error: 8.0425e-14
Epoch 420/500
34/34 [==============================] - 0s - loss: 8.3096e-14 - mean_squared_error: 8.3096e-14
Epoch 421/500
34/34 [==============================] - 0s - loss: 7.7917e-14 - mean_squared_error: 7.7917e-14
Epoch 422/500
34/34 [===

34/34 [==============================] - 0s - loss: 4.9427e-14 - mean_squared_error: 4.9427e-14
Epoch 488/500
34/34 [==============================] - 0s - loss: 4.7226e-14 - mean_squared_error: 4.7226e-14
Epoch 489/500
34/34 [==============================] - 0s - loss: 5.3580e-14 - mean_squared_error: 5.3580e-14
Epoch 490/500
34/34 [==============================] - 0s - loss: 5.4788e-14 - mean_squared_error: 5.4788e-14
Epoch 491/500
34/34 [==============================] - 0s - loss: 5.3195e-14 - mean_squared_error: 5.3195e-14
Epoch 492/500
34/34 [==============================] - 0s - loss: 5.3756e-14 - mean_squared_error: 5.3756e-14
Epoch 493/500
34/34 [==============================] - 0s - loss: 2.1371e-14 - mean_squared_error: 2.1371e-14
Epoch 494/500
34/34 [==============================] - 0s - loss: 2.4930e-14 - mean_squared_error: 2.4930e-14
Epoch 495/500
34/34 [==============================] - 0s - loss: 2.6262e-14 - mean_squared_error: 2.6262e-14
Epoch 496/500
34/34 [===

### Backward Ensemble

In [10]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

Training Ensemble 0 - 

Training data -  1
Shapes - 
 X = (34, 30) 
 Y = (34, 1)
Epoch 1/500
34/34 [==============================] - 1s - loss: 1.6915 - mean_squared_error: 1.6915
Epoch 2/500
34/34 [==============================] - 0s - loss: 1.3993 - mean_squared_error: 1.3993
Epoch 3/500
34/34 [==============================] - 0s - loss: 1.2151 - mean_squared_error: 1.2151
Epoch 4/500
34/34 [==============================] - 0s - loss: 1.0961 - mean_squared_error: 1.0961
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.9990 - mean_squared_error: 0.9990
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.9020 - mean_squared_error: 0.9020
Epoch 7/500
34/34 [==============================] - 0s - loss: 0.8028 - mean_squared_error: 0.8028
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.7085 - mean_squared_error: 0.7085
Epoch 9/500
34/34 [==============================] - 0s - loss: 0.6259 - mean_squared_error: 0.6259
Epoch 10/500
34/34 

34/34 [==============================] - 0s - loss: 0.0182 - mean_squared_error: 0.0182
Epoch 82/500
34/34 [==============================] - 0s - loss: 0.0179 - mean_squared_error: 0.0179
Epoch 83/500
34/34 [==============================] - 0s - loss: 0.0175 - mean_squared_error: 0.0175
Epoch 84/500
34/34 [==============================] - 0s - loss: 0.0171 - mean_squared_error: 0.0171
Epoch 85/500
34/34 [==============================] - 0s - loss: 0.0167 - mean_squared_error: 0.0167
Epoch 86/500
34/34 [==============================] - 0s - loss: 0.0163 - mean_squared_error: 0.0163
Epoch 87/500
34/34 [==============================] - 0s - loss: 0.0159 - mean_squared_error: 0.0159
Epoch 88/500
34/34 [==============================] - 0s - loss: 0.0155 - mean_squared_error: 0.0155
Epoch 89/500
34/34 [==============================] - 0s - loss: 0.0151 - mean_squared_error: 0.0151
Epoch 90/500
34/34 [==============================] - 0s - loss: 0.0149 - mean_squared_error: 0.0149
Epo

34/34 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 162/500
34/34 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 163/500
34/34 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 164/500
34/34 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 165/500
34/34 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 166/500
34/34 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 167/500
34/34 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 168/500
34/34 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 169/500
34/34 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 170/500
34/34 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0

34/34 [==============================] - 0s - loss: 1.1453e-04 - mean_squared_error: 1.1453e-04
Epoch 238/500
34/34 [==============================] - 0s - loss: 1.0938e-04 - mean_squared_error: 1.0938e-04
Epoch 239/500
34/34 [==============================] - 0s - loss: 1.0420e-04 - mean_squared_error: 1.0420e-04
Epoch 240/500
34/34 [==============================] - 0s - loss: 9.8545e-05 - mean_squared_error: 9.8545e-05
Epoch 241/500
34/34 [==============================] - 0s - loss: 9.2919e-05 - mean_squared_error: 9.2919e-05
Epoch 242/500
34/34 [==============================] - 0s - loss: 8.7856e-05 - mean_squared_error: 8.7856e-05
Epoch 243/500
34/34 [==============================] - 0s - loss: 8.3504e-05 - mean_squared_error: 8.3504e-05
Epoch 244/500
34/34 [==============================] - 0s - loss: 7.9351e-05 - mean_squared_error: 7.9351e-05
Epoch 245/500
34/34 [==============================] - 0s - loss: 7.5470e-05 - mean_squared_error: 7.5470e-05
Epoch 246/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.0213e-06 - mean_squared_error: 1.0213e-06
Epoch 312/500
34/34 [==============================] - 0s - loss: 9.5100e-07 - mean_squared_error: 9.5100e-07
Epoch 313/500
34/34 [==============================] - 0s - loss: 8.7172e-07 - mean_squared_error: 8.7172e-07
Epoch 314/500
34/34 [==============================] - 0s - loss: 8.0203e-07 - mean_squared_error: 8.0203e-07
Epoch 315/500
34/34 [==============================] - 0s - loss: 7.4252e-07 - mean_squared_error: 7.4252e-07
Epoch 316/500
34/34 [==============================] - 0s - loss: 6.9384e-07 - mean_squared_error: 6.9384e-07
Epoch 317/500
34/34 [==============================] - 0s - loss: 6.4634e-07 - mean_squared_error: 6.4634e-07
Epoch 318/500
34/34 [==============================] - 0s - loss: 5.9462e-07 - mean_squared_error: 5.9462e-07
Epoch 319/500
34/34 [==============================] - 0s - loss: 5.4312e-07 - mean_squared_error: 5.4312e-07
Epoch 320/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 3.3556e-04 - mean_squared_error: 3.3556e-04
Epoch 136/500
34/34 [==============================] - 0s - loss: 3.2403e-04 - mean_squared_error: 3.2403e-04
Epoch 137/500
34/34 [==============================] - 0s - loss: 3.1276e-04 - mean_squared_error: 3.1276e-04
Epoch 138/500
34/34 [==============================] - 0s - loss: 3.0176e-04 - mean_squared_error: 3.0176e-04
Epoch 139/500
34/34 [==============================] - 0s - loss: 2.9133e-04 - mean_squared_error: 2.9133e-04
Epoch 140/500
34/34 [==============================] - 0s - loss: 2.8142e-04 - mean_squared_error: 2.8142e-04
Epoch 141/500
34/34 [==============================] - 0s - loss: 2.7131e-04 - mean_squared_error: 2.7131e-04
Epoch 142/500
34/34 [==============================] - 0s - loss: 2.6161e-04 - mean_squared_error: 2.6161e-04
Epoch 143/500
34/34 [==============================] - 0s - loss: 2.5246e-04 - mean_squared_error: 2.5246e-04
Epoch 144/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.1695e-05 - mean_squared_error: 2.1695e-05
Epoch 210/500
34/34 [==============================] - 0s - loss: 2.0807e-05 - mean_squared_error: 2.0807e-05
Epoch 211/500
34/34 [==============================] - 0s - loss: 1.9944e-05 - mean_squared_error: 1.9944e-05
Epoch 212/500
34/34 [==============================] - 0s - loss: 1.9118e-05 - mean_squared_error: 1.9118e-05
Epoch 213/500
34/34 [==============================] - 0s - loss: 1.8355e-05 - mean_squared_error: 1.8355e-05
Epoch 214/500
34/34 [==============================] - 0s - loss: 1.7589e-05 - mean_squared_error: 1.7589e-05
Epoch 215/500
34/34 [==============================] - 0s - loss: 1.6846e-05 - mean_squared_error: 1.6846e-05
Epoch 216/500
34/34 [==============================] - 0s - loss: 1.6169e-05 - mean_squared_error: 1.6169e-05
Epoch 217/500
34/34 [==============================] - 0s - loss: 1.5513e-05 - mean_squared_error: 1.5513e-05
Epoch 218/500
34/34 [===

34/34 [==============================] - 0s - loss: 8.0268e-07 - mean_squared_error: 8.0268e-07
Epoch 284/500
34/34 [==============================] - 0s - loss: 7.6858e-07 - mean_squared_error: 7.6858e-07
Epoch 285/500
34/34 [==============================] - 0s - loss: 7.3458e-07 - mean_squared_error: 7.3458e-07
Epoch 286/500
34/34 [==============================] - 0s - loss: 7.0500e-07 - mean_squared_error: 7.0500e-07
Epoch 287/500
34/34 [==============================] - 0s - loss: 6.7645e-07 - mean_squared_error: 6.7645e-07
Epoch 288/500
34/34 [==============================] - 0s - loss: 6.4765e-07 - mean_squared_error: 6.4765e-07
Epoch 289/500
34/34 [==============================] - 0s - loss: 6.1909e-07 - mean_squared_error: 6.1909e-07
Epoch 290/500
34/34 [==============================] - 0s - loss: 5.9271e-07 - mean_squared_error: 5.9271e-07
Epoch 291/500
34/34 [==============================] - 0s - loss: 5.6836e-07 - mean_squared_error: 5.6836e-07
Epoch 292/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.7840e-08 - mean_squared_error: 2.7840e-08
Epoch 358/500
34/34 [==============================] - 0s - loss: 2.6568e-08 - mean_squared_error: 2.6568e-08
Epoch 359/500
34/34 [==============================] - 0s - loss: 2.5320e-08 - mean_squared_error: 2.5320e-08
Epoch 360/500
34/34 [==============================] - 0s - loss: 2.4121e-08 - mean_squared_error: 2.4121e-08
Epoch 361/500
34/34 [==============================] - 0s - loss: 2.2878e-08 - mean_squared_error: 2.2878e-08
Epoch 362/500
34/34 [==============================] - 0s - loss: 2.1785e-08 - mean_squared_error: 2.1785e-08
Epoch 363/500
34/34 [==============================] - 0s - loss: 2.0788e-08 - mean_squared_error: 2.0788e-08
Epoch 364/500
34/34 [==============================] - 0s - loss: 1.9775e-08 - mean_squared_error: 1.9775e-08
Epoch 365/500
34/34 [==============================] - 0s - loss: 1.8811e-08 - mean_squared_error: 1.8811e-08
Epoch 366/500
34/34 [===

34/34 [==============================] - 0s - loss: 5.3473e-10 - mean_squared_error: 5.3473e-10
Epoch 432/500
34/34 [==============================] - 0s - loss: 5.0152e-10 - mean_squared_error: 5.0152e-10
Epoch 433/500
34/34 [==============================] - 0s - loss: 4.7737e-10 - mean_squared_error: 4.7737e-10
Epoch 434/500
34/34 [==============================] - 0s - loss: 4.5123e-10 - mean_squared_error: 4.5123e-10
Epoch 435/500
34/34 [==============================] - 0s - loss: 4.2345e-10 - mean_squared_error: 4.2345e-10
Epoch 436/500
34/34 [==============================] - 0s - loss: 4.0145e-10 - mean_squared_error: 4.0145e-10
Epoch 437/500
34/34 [==============================] - 0s - loss: 3.8028e-10 - mean_squared_error: 3.8028e-10
Epoch 438/500
34/34 [==============================] - 0s - loss: 3.5574e-10 - mean_squared_error: 3.5574e-10
Epoch 439/500
34/34 [==============================] - 0s - loss: 3.3578e-10 - mean_squared_error: 3.3578e-10
Epoch 440/500
34/34 [===

### Perceptron fuzzifier

### Training the fuzzifier

In [11]:
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print('\n Ensemble {} - \n'.format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    
    j = 0
    
    y_pred_fw = []
    y_pred_re = []
    
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        
        print('Getting op from chain node ',j)
        node_name = 'Node' + str(i)
        print('Shapes - \n X = {}'.format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)


 Ensemble 1 - 

Getting op from chain node  0
Shapes - 
 X = (34, 30)
Getting op from chain node  1
Shapes - 
 X = (34, 31)
Getting op from chain node  2
Shapes - 
 X = (34, 32)
Getting op from chain node  3
Shapes - 
 X = (34, 33)
Getting op from chain node  4
Shapes - 
 X = (34, 34)
Getting op from chain node  5
Shapes - 
 X = (34, 35)


In [12]:
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],np.array(y_full_fw).shape[2])
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],np.array(y_full_re).shape[2])

In [13]:
forward_merged[:,0]

array([ 0.4951803 ,  0.46117845,  1.124648  ,  1.0613569 , -0.11275344,
       -0.27728093], dtype=float32)

In [14]:
full_set = np.append(forward_merged,reverse_merged,axis=0).T
full_set.shape

(34, 12)

### Creating FCNN fuzzifier

In [15]:
fuzzifier = Sequential()
fuzzifier.add(Dense(y_train.shape[1],activation='linear',input_dim = full_set.shape[1]))
fuzzifier.compile(optimizer='adam',loss='mse')
fuzzifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 6)                 78        
Total params: 78
Trainable params: 78
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Training Fuzzifier
fuzzifier.fit(full_set,y_train,epochs=500,batch_size=5)

Epoch 1/500
34/34 [==============================] - 1s - loss: 3.0525     
Epoch 2/500
34/34 [==============================] - 0s - loss: 2.9509     
Epoch 3/500
34/34 [==============================] - 0s - loss: 2.8457     
Epoch 4/500
34/34 [==============================] - 0s - loss: 2.7485     
Epoch 5/500
34/34 [==============================] - 0s - loss: 2.6496     
Epoch 6/500
34/34 [==============================] - 0s - loss: 2.5600     
Epoch 7/500
34/34 [==============================] - 0s - loss: 2.4706     
Epoch 8/500
34/34 [==============================] - 0s - loss: 2.3810     
Epoch 9/500
34/34 [==============================] - 0s - loss: 2.3004     
Epoch 10/500
34/34 [==============================] - 0s - loss: 2.2177     
Epoch 11/500
34/34 [==============================] - 0s - loss: 2.1431     
Epoch 12/500
34/34 [==============================] - 0s - loss: 2.0727     
Epoch 13/500
34/34 [==============================] - 0s - loss: 1.9933     
Epoch 14

34/34 [==============================] - 0s - loss: 0.0706     
Epoch 108/500
34/34 [==============================] - 0s - loss: 0.0683     
Epoch 109/500
34/34 [==============================] - 0s - loss: 0.0659     
Epoch 110/500
34/34 [==============================] - 0s - loss: 0.0639     
Epoch 111/500
34/34 [==============================] - 0s - loss: 0.0617     
Epoch 112/500
34/34 [==============================] - 0s - loss: 0.0597     
Epoch 113/500
34/34 [==============================] - 0s - loss: 0.0577     
Epoch 114/500
34/34 [==============================] - 0s - loss: 0.0559     
Epoch 115/500
34/34 [==============================] - 0s - loss: 0.0540     
Epoch 116/500
34/34 [==============================] - 0s - loss: 0.0524     
Epoch 117/500
34/34 [==============================] - 0s - loss: 0.0506     
Epoch 118/500
34/34 [==============================] - 0s - loss: 0.0491     
Epoch 119/500
34/34 [==============================] - 0s - loss: 0.0476     


34/34 [==============================] - 0s - loss: 0.0071     
Epoch 212/500
34/34 [==============================] - 0s - loss: 0.0070     
Epoch 213/500
34/34 [==============================] - 0s - loss: 0.0069     
Epoch 214/500
34/34 [==============================] - 0s - loss: 0.0068     
Epoch 215/500
34/34 [==============================] - 0s - loss: 0.0067     
Epoch 216/500
34/34 [==============================] - 0s - loss: 0.0066     
Epoch 217/500
34/34 [==============================] - 0s - loss: 0.0065     
Epoch 218/500
34/34 [==============================] - 0s - loss: 0.0065     
Epoch 219/500
34/34 [==============================] - 0s - loss: 0.0064     
Epoch 220/500
34/34 [==============================] - 0s - loss: 0.0063     
Epoch 221/500
34/34 [==============================] - 0s - loss: 0.0062     
Epoch 222/500
34/34 [==============================] - 0s - loss: 0.0062     
Epoch 223/500
34/34 [==============================] - 0s - loss: 0.0061     


34/34 [==============================] - 0s - loss: 0.0024     
Epoch 317/500
34/34 [==============================] - 0s - loss: 0.0024     
Epoch 318/500
34/34 [==============================] - 0s - loss: 0.0024     
Epoch 319/500
34/34 [==============================] - 0s - loss: 0.0024     
Epoch 320/500
34/34 [==============================] - 0s - loss: 0.0023     
Epoch 321/500
34/34 [==============================] - 0s - loss: 0.0023     
Epoch 322/500
34/34 [==============================] - 0s - loss: 0.0023     
Epoch 323/500
34/34 [==============================] - ETA: 0s - loss: 0.002 - 0s - loss: 0.0023     
Epoch 324/500
34/34 [==============================] - 0s - loss: 0.0023     
Epoch 325/500
34/34 [==============================] - 0s - loss: 0.0023     
Epoch 326/500
34/34 [==============================] - 0s - loss: 0.0022     
Epoch 327/500
34/34 [==============================] - 0s - loss: 0.0022     
Epoch 328/500
34/34 [==============================] -

34/34 [==============================] - 0s - loss: 0.0015     
Epoch 421/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 422/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 423/500
34/34 [==============================] - 0s - loss: 0.0015         
Epoch 424/500
34/34 [==============================] - 0s - loss: 0.0015         
Epoch 425/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 426/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 427/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 428/500
34/34 [==============================] - 0s - loss: 0.0015         
Epoch 429/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 430/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 431/500
34/34 [==============================] - 0s - loss: 0.0015     
Epoch 432/500
34/34 [==============================] - 0s - loss: 

### Testing FCNN fuzzifier

In [17]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
        
## Merged Ensemble Reverse
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],
                           np.array(y_full_fw).shape[2])

## Merged Ensemble Reverse
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],
                           np.array(y_full_re).shape[2])

full_set_test = np.append(forward_merged,reverse_merged,axis=0).T
print("\n\nTest shape:, ",full_set_test.shape)



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


Test shape:,  (15, 12)


In [18]:
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(15, 6)


In [19]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  3.864974808558809
MAE:  1.7702864884270564
R_Squared:  0.7863264432649381
Adjusted R-squared -  0.4302038487065015


### Convolutional Fuzzifier

In [20]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (34, 30)
Getting op from chain node  1
Shapes:
 X = (34, 31)
Getting op from chain node  2
Shapes:
 X = (34, 32)
Getting op from chain node  3
Shapes:
 X = (34, 33)
Getting op from chain node  4
Shapes:
 X = (34, 34)
Getting op from chain node  5
Shapes:
 X = (34, 35)


In [21]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [22]:
full_set = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set = np.append(full_set,intervened[i,:,:],axis = 1)

In [23]:
from keras.layers import Conv1D,Flatten
#CNN Fuzzifier 
fuzzifier = Sequential()
fuzzifier.add(Conv1D(filters=1,kernel_size=2,strides=2,activation='relu',input_shape=(full_set.shape[1],1)))
fuzzifier.add(Flatten())
fuzzifier.add(Dense(128,activation='relu'))
fuzzifier.add(Dense(y_train.shape[1],activation='linear'))
fuzzifier.compile(optimizer='adam',loss='mse')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [24]:
fuzzifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 6, 1)              3         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 128)               896       
_________________________________________________________________
dense_39 (Dense)             (None, 6)                 774       
Total params: 1,673
Trainable params: 1,673
Non-trainable params: 0
_________________________________________________________________


In [25]:
#Training Fuzzifier
fuzzifier.fit(full_set.reshape(full_set.shape[0],full_set.shape[1],1),y_train,epochs=500,batch_size=5)

Epoch 1/500
34/34 [==============================] - 1s - loss: 1.0215     
Epoch 2/500
34/34 [==============================] - 0s - loss: 0.9731     
Epoch 3/500
34/34 [==============================] - 0s - loss: 0.9292     
Epoch 4/500
34/34 [==============================] - 0s - loss: 0.8831     
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.8335     
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.7801     
Epoch 7/500
34/34 [==============================] - ETA: 0s - loss: 0.285 - 0s - loss: 0.7236     
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.6661     
Epoch 9/500
34/34 [==============================] - 0s - loss: 0.6102     
Epoch 10/500
34/34 [==============================] - 0s - loss: 0.5517     
Epoch 11/500
34/34 [==============================] - 0s - loss: 0.5000     
Epoch 12/500
34/34 [==============================] - 0s - loss: 0.4450     
Epoch 13/500
34/34 [==============================] - 0s - lo

34/34 [==============================] - 0s - loss: 0.0218     
Epoch 107/500
34/34 [==============================] - 0s - loss: 0.0216     
Epoch 108/500
34/34 [==============================] - 0s - loss: 0.0217     
Epoch 109/500
34/34 [==============================] - 0s - loss: 0.0220     
Epoch 110/500
34/34 [==============================] - 0s - loss: 0.0213     
Epoch 111/500
34/34 [==============================] - 0s - loss: 0.0211     
Epoch 112/500
34/34 [==============================] - 0s - loss: 0.0209     
Epoch 113/500
34/34 [==============================] - 0s - loss: 0.0209     
Epoch 114/500
34/34 [==============================] - 0s - loss: 0.0204     
Epoch 115/500
34/34 [==============================] - 0s - loss: 0.0202     
Epoch 116/500
34/34 [==============================] - 0s - loss: 0.0199     
Epoch 117/500
34/34 [==============================] - 0s - loss: 0.0199     
Epoch 118/500
34/34 [==============================] - 0s - loss: 0.0195     


34/34 [==============================] - 0s - loss: 0.0022     
Epoch 211/500
34/34 [==============================] - 0s - loss: 0.0022         
Epoch 212/500
34/34 [==============================] - 0s - loss: 0.0021     
Epoch 213/500
34/34 [==============================] - 0s - loss: 0.0021     
Epoch 214/500
34/34 [==============================] - 0s - loss: 0.0022     
Epoch 215/500
34/34 [==============================] - 0s - loss: 0.0021     
Epoch 216/500
34/34 [==============================] - 0s - loss: 0.0020     
Epoch 217/500
34/34 [==============================] - 0s - loss: 0.0021     
Epoch 218/500
34/34 [==============================] - 0s - loss: 0.0020     
Epoch 219/500
34/34 [==============================] - 0s - loss: 0.0020     
Epoch 220/500
34/34 [==============================] - 0s - loss: 0.0020     
Epoch 221/500
34/34 [==============================] - 0s - loss: 0.0020     
Epoch 222/500
34/34 [==============================] - 0s - loss: 0.0019  

34/34 [==============================] - 0s - loss: 0.0014     
Epoch 314/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 315/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 316/500
34/34 [==============================] - 0s - loss: 0.0014     
Epoch 317/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 318/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 319/500
34/34 [==============================] - 0s - loss: 0.0014     
Epoch 320/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 321/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 322/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 323/500
34/34 [==============================] - 0s - loss: 0.0014     
Epoch 324/500
34/34 [==============================] - 0s - loss: 0.0014     
Epoch 325/500
34/34 [=============================

34/34 [==============================] - 0s - loss: 0.0013     
Epoch 415/500
34/34 [==============================] - 0s - loss: 0.0013     
Epoch 416/500
34/34 [==============================] - 0s - loss: 0.0014         
Epoch 417/500
34/34 [==============================] - 0s - loss: 0.0013     
Epoch 418/500
34/34 [==============================] - 0s - loss: 0.0013         
Epoch 419/500
34/34 [==============================] - 0s - loss: 0.0013         
Epoch 420/500
34/34 [==============================] - 0s - loss: 0.0012     
Epoch 421/500
34/34 [==============================] - 0s - loss: 0.0012         
Epoch 422/500
34/34 [==============================] - 0s - loss: 0.0012         
Epoch 423/500
34/34 [==============================] - 0s - loss: 0.0012         
Epoch 424/500
34/34 [==============================] - 0s - loss: 0.0012         
Epoch 425/500
34/34 [==============================] - 0s - loss: 0.0013     
Epoch 426/500
34/34 [=============================

### Testing CNN Fuzzifier

In [26]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


In [27]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [28]:
full_set_test = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set_test = np.append(full_set_test,intervened[i,:,:],axis = 1)

In [29]:
full_set_test = full_set_test.reshape(full_set_test.shape[0],full_set_test.shape[1],1)
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(15, 6)


In [30]:
evaluate(scaler.inverse_transform(y_test),
         scaler.inverse_transform(np.array(predictions)))

RMSE:  9.741039795277644
MAE:  4.329993569268121
R_Squared:  -0.008040643964744057
Adjusted R-squared -  -1.688108383905984


### Mean Aggregation

In [31]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (15, 30)
Getting op from chain node  1
Shapes:
 X = (15, 31)
Getting op from chain node  2
Shapes:
 X = (15, 32)
Getting op from chain node  3
Shapes:
 X = (15, 33)
Getting op from chain node  4
Shapes:
 X = (15, 34)
Getting op from chain node  5
Shapes:
 X = (15, 35)


In [32]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [33]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [34]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  3.7051202266927104
MAE:  1.5997109685473976
R_Squared:  0.8172138481323025
Adjusted R-squared -  0.5125702616861398
